ノーマルのニューラルネットワーク

In [ ]:
# 分かち書きを行うためにjanomeというPythonパッケージをインストール
!pip install janome

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.7 MB 1.3 MB/s 


In [ ]:
import pandas as pd # CSVを含む表形式データを読み込み，分析するためのパッケージ

# CSVファイルを読み込み
dataset = pd.read_csv('drive/MyDrive/実習B/full_AI.csv')
dataset

,id,label,text
0,1,1,One day I noticed. I gatta go to that stupid s...
1,2,1,全てを嘘から I shall change myself！冒険に出たあの2009「I bel...
2,3,1,ずっと笑っていられるんだ大袈裟なんかじゃないこれからも一緒に歌ってくれ一人一人違う声で犬も吠...
3,4,1,We can find it暴くのさ真実を今夜待ちわびた世界なら探してた言葉さえ誰にも邪魔さ...
4,5,1,Oh oh oh(Rewind)We can't stop it…When the time...
...,...,...,...
1195,1196,0,スモークとコカインやらゴールドのチェーンいらず一ひとつの個性こせいでhip hopと呼よべる...
1196,1197,0,ボヤけた姿すがた 朝靄あさもやの向むこう側がわ大事だいじに頂いただく 冷さめた御飯ごはん見慣...
1197,1198,0,ugh ughstop thinkin bout itstop thinkin bout i...
1198,1199,0,'cause I'm singing “Alright” …アラームで起おきたTV 流ながし...


In [ ]:
from janome.tokenizer import Tokenizer # janomeからTokenizerクラスをimportする

# janomeはPythonで書かれた形態素解析器であり単語の分かち書きなどが可能
# なお，速度に難があるため，大量のテキストを処理するときには，MeCabを利用すること
tokenizer = Tokenizer(wakati=True)
# text列の文字列を単語に分割し保存する
dataset['text'] = dataset['text'].map(lambda x: list(tokenizer.tokenize(x)))
dataset

,id,label,text
0,1,1,"[One, , day, , I, , noticed, ., , I, , ga..."
1,2,1,"[全て, を, 嘘, から, , I, , shall, , change, , m..."
2,3,1,"[ずっと, 笑っ, て, い, られる, ん, だ, 大袈裟, な, ん, か, じゃ, な..."
3,4,1,"[We, , can, , find, , it, 暴く, の, さ, 真実, を, ..."
4,5,1,"[Oh, , oh, , oh, (, Rewind, ), We, , can, '..."
...,...,...,...
1195,1196,0,"[スモーク, と, コカイン, やら, ゴールド, の, チェーン, いら, ず, 一, ひ..."
1196,1197,0,"[ボヤけ, た, 姿, す, がた, , 朝靄, あ, さも, や, の, 向, むこ, ..."
1197,1198,0,"[ugh, , ughstop, , thinkin, , bout, , itst..."
1198,1199,0,"[', cause, , I, ', m, , singing, , “, Alrig..."


In [ ]:
from torchtext.vocab import build_vocab_from_iterator
import torchtext.transforms as T

# 単語辞書作成
text_vocab = build_vocab_from_iterator(dataset['text'], specials=('<unk>', '<pad>'))

# テキストの変換方法を定義
text_transform = T.Sequential(
    T.VocabTransform(text_vocab), # 単語を整数値に置き換え
    T.ToTensor(padding_value=text_vocab['<pad>']) # <pad>によって長さを揃える
)

stoi = text_vocab.get_stoi() # 各単語に割り当てられた整数値を取得（str to int）

In [ ]:
import torch
from torch.utils.data import DataLoader

train_size = 1000 # 先頭の20,000件を訓練データとして用いることにする
batch_size = 10 # ミニバッチのサイズを設定

def collate_batch(batch):
    # batchには，CSVの複数行（batch_sizeで指定している）がリストとして渡される．各行は[id, label, text]という形式である．
    texts = text_transform([text for (id, label, text) in batch]) # 定義したテキスト変換を適用
    labels = torch.Tensor([label for (id, label, text) in batch]) # ラベルは整数値なのでそのまま利用する．ただし，Tensor型というPyTorchで扱える型にしておく．ラベルが文字列などの場合は整数値に置き換える．
    return texts, labels

# 読み込みんだCSVのうち，先頭の `train_size` 件を訓練データ，残りをテストデータとして用いる
train_dataset, test_dataset = dataset[:train_size], dataset[train_size:]

# 訓練データ，および，テストデータ用のデータローダを用意する
train_data_loader = DataLoader(train_dataset.values, batch_size=batch_size, shuffle=True, collate_fn=collate_batch)
test_data_loader = DataLoader(test_dataset.values, batch_size=batch_size, shuffle=False, collate_fn=collate_batch)

In [ ]:
for i, (texts, labels) in enumerate(train_data_loader):
    print(i)
    for text, label in zip(texts, labels):
        print(text, label)
    break # 今回の例では1つのミニバッチを取り出したら強制的にループを抜けている

0
tensor([  562,     5,   659,    22,    46, 20117,    97,     3,    85,    11,
           76,    23,    12,   104,    70,    10,    46,   952,    12,   104,
           70,    10,  3985,  1446,   487,     9,   658,    46,  1446,    70,
            9,    81,     3,    18, 17870,    15, 20528,     5,  6227,     5,
           75,   232,    60,     3,  2805,  3112,    24, 21061,     7,    19,
           17,    22,  9753,    79,  9774,    43,     7,    22,  6743,   128,
           97,   202,    99,    11,   109, 21136,   138,  3097,    55,  2212,
           11,  2918,     4,   304,    44,   120,   229,    83,     6,    94,
           83,     6,    71,     4,    10,  4252,    12,   115,     6,  2440,
          401,     3,    85,    11, 17741,  5227,     3,   662,    11, 10080,
        15016,  4996, 19114,   249,     4, 21160,    83,     8,    71,    18,
         7421,     5,   399,     5,    71,    10,    23,    12,  9320,    15,
          307,     6,  1649,     5,    75,   150,  2907,    14

In [ ]:
# ここではfastTextという単語埋め込みを利用する
from torchtext.vocab import FastText
fasttext = FastText(language="ja")

# fastTextも「辞書」を持っており，例えば，「の」という単語には整数の2が対応しており，
# さらに300次元の単語埋め込みが既に用意されている．
print(fasttext.itos[2], 2, fasttext.vectors[0])

.vector_cache/wiki.ja.vec: 1.37GB [00:26, 52.5MB/s]                            
100%|██████████| 580000/580000 [01:17<00:00, 7508.45it/s] 


の 2 tensor([  2.4405,  -2.4237,  -2.3451, -12.3030,  -0.8740,  -1.8989,   2.7567,
          1.6199,   5.1494,  -4.1123,   0.3489,   4.1822,  -2.4252,  -3.2331,
          3.3351,  -1.1762,  -7.0971,  -3.4565,  -4.3095,   3.2371,   2.3855,
         -4.2671,  -2.7152,  -1.1240,   2.2580,  -6.9810,   4.3527,   2.1409,
          2.7580,  -6.1698,  -0.2339,  -1.7493,  -6.9762,  -8.5031,  -5.2302,
         -3.9934,  -4.2426,  -8.5977,   7.3990,  -4.9724,   2.7592,   4.1365,
          2.8621,  -4.2423,  -4.2332,  -2.1560,   1.9182,   3.3009,   2.8669,
          2.0220,  -5.0652,   3.4419,  -0.9964,   7.1275,   2.0687,  -1.2856,
         -1.4175,   4.1298,  -4.2781,   3.2888,   0.4101,   2.5506,  -0.1357,
          4.2976,  -3.6452,  -1.2767,   4.8675,   4.6114,   9.9612,  -6.9417,
          3.6101,   4.2744,  -6.4286,   6.0007,  -6.1865,   0.4312,   0.8587,
          3.3407,  -4.7399,   5.2652,  -2.9183,   7.4144,   2.6888,  17.5140,
          0.7445,   0.7603,   3.0601,   5.2463,  -2.9451,  -

In [ ]:
import torch
from torchtext.vocab import vocab

# 先程までは，自前の「辞書」を作成していたが，fastTextの辞書に切り替えるためにいくつか前処理を行う
# fastTextの辞書にない単語や<pad>などには，何も意味を持たないようなすべてが0であるベクトルを割り当てるようにする．
# このため，辞書のデフォルトインデックスを辞書の(一番最後の整数値+1)を割り当て，これがゼロベクトルになるようにする．
fasttext_vocab = vocab(fasttext.stoi)
special_token_id = len(fasttext)
fasttext_vocab.set_default_index(special_token_id)
shape = fasttext.vectors.shape
fasttext_vectors = torch.zeros((shape[0]+1, shape[1]))
fasttext_vectors[:shape[0]] = fasttext.vectors

# テキストの変換方法を再定義
text_transform = T.Sequential(
    T.VocabTransform(fasttext_vocab), # 単語をfastTextの辞書で置き換え
    T.ToTensor(padding_value=special_token_id) # (一番最後の整数値+1)という整数値で長さを揃える
)

# text_transformが変更されているため再定義
def collate_batch(batch):
    texts = text_transform([text for (id, label, text) in batch])
    labels = torch.Tensor([label for (id, label, text) in batch])
    return texts, labels

# 訓練データ，および，テストデータ用のデータローダを再度用意する
train_data_loader = DataLoader(train_dataset.values, batch_size=batch_size, shuffle=True, collate_fn=collate_batch)
test_data_loader = DataLoader(test_dataset.values, batch_size=batch_size, shuffle=False, collate_fn=collate_batch)

# 各単語に整数値が割り当てられ，また，整数値ごとに埋め込みが得られていることを確認
for i, (texts, labels) in enumerate(train_data_loader):
    for text, label in zip(texts, labels):
        print(text, label)
        for t in text[:10]: # 最初の10単語分だけ埋め込みを表示
          print(fasttext_vectors[t][:5]) # 300次元中の最初の5次元の値だけ表示
        break
    break # 1テキストだけで終了

tensor([580000,     20,     49,  ..., 580000, 580000, 580000]) tensor(1.)
tensor([0., 0., 0., 0., 0.])
tensor([  2.5735,  -2.2460,  -2.3734, -12.5430,  -0.9702])
tensor([  2.4404,  -2.2515,  -2.5030, -12.5360,  -0.8457])
tensor([  3.1573,  -2.1580,  -2.4815, -13.0130,  -0.9487])
tensor([ 1.5283, -1.5611, -1.3456, -8.1991, -0.4351])
tensor([ 1.2894, -1.4129, -1.2643, -5.5472, -0.4812])
tensor([ 1.0757, -2.3466, -0.1838, -9.8013, -0.1952])
tensor([0., 0., 0., 0., 0.])
tensor([ 1.0606, -0.8845, -1.1295, -5.2311, -0.3358])
tensor([  2.4827,  -2.5240,  -2.4796, -12.0690,  -1.3923])


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TextClassifier(nn.Module):
    def __init__(self):
        super(TextClassifier, self).__init__()

        # 単語埋め込み（FastText）を設定する
        self.embedding = torch.nn.Embedding.from_pretrained(
            embeddings=fasttext_vectors, freeze=False)
        
        emb_size = fasttext_vectors.shape[1]
        self.fc1 = nn.Linear(emb_size, 100) # 100 x 300 の行列を含む全結合層を設定
        self.fc2 = nn.Linear(100, 1) # 1 x 100 の行列（この場合はベクトル）を含む全結合層を設定

    def forward(self, seq):
        x = self.embedding(seq) # 各単語を単語埋め込みに変換する
        x = x.mean(axis=1) # 各テキストに含まれるすべての単語の埋め込みの平均をとる
        x = F.relu(self.fc1(x))
        h = self.fc2(x)
        h = h.squeeze()
        return h

In [ ]:
import torch
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # CPUもしくはGPUのどちらを使うかを設定
model = TextClassifier() # ニューラルネットワークモデルのインスタンスを生成
model = model.to(device) # CPUもしくはGPUのどちらを設定
optimizer = optim.Adam(model.parameters()) # 基本的な学習方法はミニバッチ勾配降下法ではあるが，その中でもよく用いられるAdamと呼ばれる方法を用いることにする

criterion = nn.BCEWithLogitsLoss() # 二値分類用の交差エントロピーを最小化することにする

epoch_size = 10 # 勾配降下法はすべてのデータでパラメータを更新したら終わりではなく，全データでの更新（=1エポック）を複数回行う必要がある

model.train() # モデルを学習モードに変更

# `epoch_size`の数だけ以下を繰り返す
for epoch in range(epoch_size):
    losses = []
    # イテレータはミニバッチ勾配降下法のために，`batch_size`で指定した数ごとにデータをわけて読み込んでくれる．
    for batch_idx, (texts, labels) in enumerate(train_data_loader):
        texts, labels = texts.to(device), labels.to(device)
        optimizer.zero_grad() # 勾配の初期化
        y = model(texts) # 現時点でのモデルの出力を得る
        loss = criterion(y, labels.type(torch.float)) # 交差エントロピーの計算
        loss.backward() # 交差エントロピーの勾配計算
        optimizer.step() # パラメータ更新
        losses.append(loss.item())

    # 現在の交差エントロピーを出力
    print('Epoch: {}\tCross Entropy: {:.6f}'.format(epoch, sum(losses)))


Epoch: 0	Cross Entropy: 65.127749
Epoch: 1	Cross Entropy: 41.803218
Epoch: 2	Cross Entropy: 24.059353
Epoch: 3	Cross Entropy: 31.302578
Epoch: 4	Cross Entropy: 18.250035
Epoch: 5	Cross Entropy: 21.821112
Epoch: 6	Cross Entropy: 14.476915
Epoch: 7	Cross Entropy: 17.575704
Epoch: 8	Cross Entropy: 13.113120
Epoch: 9	Cross Entropy: 15.372465


In [ ]:
correct = 0
model.eval() # モデルを評価モードに変更
for batch_idx, (texts, labels) in enumerate(test_data_loader):
    texts, labels = texts.to(device), labels.to(device)
    y = model(texts) # モデルの出力を得る
    result = torch.sigmoid(y) # `TextClassifier`ではsigmoid関数を適用していなかったのでここで適用
    prediction = result >= 0.5 # `result`ベクトルと同じ次元を持ち，`result`の中で0.5以上である次元がTrue，それ以外がFalseであるベクトルを`prediction`とする
    target = labels == 1 # `labels`ベクトルと同じ次元を持ち，`labels`の中で1である次元がTrue，それ以外がFalseであるベクトルを`target`とする
    correct_num = target.eq(prediction).sum().item() # `prediction`ベクトルと`target`ベクトルでTrue/Falseが一致したものの数を数える
    correct += correct_num

# test_iterator.datasetにはテストデータ全体が入っているので，これの長さはテストデータの事例数となる
print("Accuracy: {:.3f}".format(correct / len(test_data_loader.dataset)))

Accuracy: 0.880
